## **The new code for handling small images (the old code was discarded).**

In [ ]:
# LIBRARY IMPORTS

# Install the ultralytics package using pip
!pip install ultralytics

# Import required libraries
from ultralytics import YOLO
from PIL import Image
import os
import json
from pathlib import Path
import numpy as np

# Without this Colab is giving an error when installing Roboflow
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Connect to google drive for accessing files.
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# HELPER FUNCTIONS ("get_next_version_path" / "convert_yolo_to_labelme" / "save_detections")

def get_next_version_path(base_path):
    """
    Get the next available version number for a directory.

    Args:
        base_path (Path): Base directory path without version number

    Returns:
        Path: New directory path with appropriate version number

    Example:
        If 'generated-labeling' exists, returns 'generated-labeling-1'
        If 'generated-labeling-1' exists too, returns 'generated-labeling-2'
    """
    if not base_path.exists():
        return base_path

    # Check for existing versioned directories
    parent = base_path.parent
    base_name = base_path.name

    # Get all existing directories that match the pattern base_name(-N)?
    existing_dirs = [d for d in parent.glob(f"{base_name}*") if d.is_dir()]

    if not existing_dirs:
        return base_path

    # Extract version numbers from existing directories
    versions = [0]  # Include 0 for the base directory
    for dir_path in existing_dirs:
        dir_name = dir_path.name
        if dir_name == base_name:
            continue
        # Try to extract version number from directory name
        try:
            version = int(dir_name.replace(f"{base_name}-", ""))
            versions.append(version)
        except ValueError:
            continue

    # Get the next version number
    next_version = max(versions) + 1
    return parent / f"{base_name}-{next_version}"

def convert_yolo_to_labelme(image_path, boxes, class_ids, class_names, confidence_scores):
    """Convert YOLO format detections to LabelMe JSON format

    Args:
        image_path (str/Path): Path to the source image
        boxes (np.array): Array of normalized bounding box coordinates [x1, y1, x2, y2]
        class_ids (np.array): Array of class IDs for each detection
        class_names (dict): Dictionary mapping class IDs to class names
        confidence_scores (np.array): Array of confidence scores for each detection

    Returns:
        dict: LabelMe format JSON dictionary, or None if conversion fails
    """
    try:
        # Open image to get dimensions
        img = Image.open(image_path)
        img_width, img_height = img.size

        # Initialize list to store shape dictionaries
        shapes = []

        # Process each detection
        for box, class_id, conf in zip(boxes, class_ids, confidence_scores):
            # Convert normalized coordinates (0-1) to absolute pixel coordinates
            x1, y1, x2, y2 = box
            x1 = float(x1 * img_width)
            y1 = float(y1 * img_height)
            x2 = float(x2 * img_width)
            y2 = float(y2 * img_height)

            # Create shape dictionary for current detection
            shape = {
                "label": class_names[int(class_id)],  # Convert class ID to name
                "points": [[x1, y1], [x2, y1], [x2, y2], [x1, y2]],  # Rectangle corners
                "group_id": None,  # Not using groups
                "shape_type": "polygon",  # Using polygon type for rectangle
                "flags": {},  # No special flags
                "confidence": float(conf)  # Add confidence score
            }
            shapes.append(shape)

        # Create full LabelMe format dictionary
        labelme_format = {
            "version": "5.1.1",  # LabelMe version
            "flags": {},  # No global flags
            "shapes": shapes,  # List of all detected objects
            "imagePath": os.path.basename(image_path),  # Just the filename
            "imageData": None,  # No embedded image data
            "imageHeight": img_height,
            "imageWidth": img_width
        }

        return labelme_format
    except Exception as e:
        print(f"Error converting to LabelMe format for {image_path}: {str(e)}")
        return None

def save_detections(model, test_images_dir, project_dir, name):
    """
    Run object detection and save results in both YOLO and LabelMe formats

    Args:
        model: Loaded YOLO model
        test_images_dir (str/Path): Directory containing test images
        project_dir (str/Path): Base directory for saving results
        name (str): Name prefix for output directories
    """
    try:
        # Create base directory path
        base_gen_lab_dir = Path(project_dir) / "generated-labeling"

        # Get next available version of the directory
        gen_lab_dir = get_next_version_path(base_gen_lab_dir)

        # Create subdirectories for YOLO and LabelMe formats
        yolo_dir = gen_lab_dir / "yolo"
        labelme_dir = gen_lab_dir / "labelme"

        # Create all required directories
        for directory in [gen_lab_dir, yolo_dir, labelme_dir]:
            directory.mkdir(parents=True, exist_ok=True)

        print(f"\nCreated new output directory: {gen_lab_dir}")

        # Convert test_images_dir to Path object
        test_path = Path(test_images_dir)
        print(f"Looking for images in: {test_path}")

        # Process each image and its detections
        image_extensions = ('.jpg', '.jpeg', '.png')
        image_files = []
        for ext in image_extensions:
            image_files.extend(list(test_path.glob(f'*{ext}')))

        if not image_files:
            raise Exception(f"No images found in {test_path}")

        print(f"Found {len(image_files)} images to process")

        # Process each image
        for image_path in image_files:
            try:
                print(f"\nProcessing image: {image_path}")
                # Get predictions for this image
                results = model(image_path)
                boxes = results[0].boxes

                # Get normalized coordinates
                coords = boxes.xyxyn.cpu().numpy()
                class_ids = boxes.cls.cpu().numpy()
                confidence_scores = boxes.conf.cpu().numpy()

                print(f"Found {len(coords)} detections")

                # Save YOLO format
                yolo_path = yolo_dir / f"{image_path.stem}.txt"
                with open(yolo_path, 'w') as f:
                    for box, class_id, conf in zip(coords, class_ids, confidence_scores):
                        # Convert to YOLO format: <class> <x_center> <y_center> <width> <height>
                        x1, y1, x2, y2 = box
                        x_center = (x1 + x2) / 2
                        y_center = (y1 + y2) / 2
                        width = x2 - x1
                        height = y2 - y1
                        f.write(f"{int(class_id)} {x_center} {y_center} {width} {height}\n")

                # Save LabelMe format
                labelme_json = convert_yolo_to_labelme(
                    image_path,
                    coords,
                    class_ids,
                    model.names,
                    confidence_scores
                )

                if labelme_json:
                    labelme_path = labelme_dir / f"{image_path.stem}.json"
                    with open(labelme_path, 'w') as f:
                        json.dump(labelme_json, f, indent=2)

                print(f"Saved results for {image_path.name}")

            except Exception as e:
                print(f"Error processing image {image_path}: {str(e)}")
                continue

        print(f"\nResults saved in:")
        print(f"YOLO format: {yolo_dir}")
        print(f"LabelMe format: {labelme_dir}")

    except Exception as e:
        print(f"Error in save_detections: {str(e)}")


In [ ]:
# CORE CODE

# Main execution block
if __name__ == "__main__":
    print("cwd:", os.getcwd())

    # Define paths
    # default address : "/content/drive/MyDrive/October15.v1i.yolov8/results" ;
    project = "/content/drive/MyDrive/October15.v1i.yolov8/results"
    # default address : "/content/drive/MyDrive/October15.v1i.yolov8/test/images" ; alt : "/content/drive/MyDrive/October15.v1i.yolov8/sampling_set/images" ;
    test_images_dir = "/content/drive/MyDrive/October15.v1i.yolov8/slicing_output"
    name = "200_epochs-"

    try:
        # Load trained model
        best_model = YOLO(f'{project}/{name}/weights/best.pt')

        # Save detections in both formats
        save_detections(
            model=best_model,
            test_images_dir=test_images_dir,
            project_dir=project,
            name=f'{name}'
        )
    except Exception as e:
        print(f"Error in main execution: {str(e)}")


In [ ]:
# SETTINGS CHECK

project = "/content/drive/MyDrive/October15.v1i.yolov8/results"
name = "200_epochs-"
best_model = YOLO(f'{project}/{name}/weights/best.pt')
print(best_model)  # This may show default settings


## **The final version of the code for handling big images (it didn't exist previously).**

In [ ]:
!pip install sahi

In [ ]:
"""
Enhanced Object Detection Script using SAHI and YOLO

Key Improvements:
- Robust error handling
- Comprehensive logging
- Flexible image processing
- Detailed performance tracking
"""

from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
import os
import json
from pathlib import Path
from PIL import Image
import numpy as np
from tqdm import tqdm
import logging
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# using an enhanced logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# versioning of directories for saving resulting outputs in the two suitable file types
def get_next_version_path(base_path):
    """
    Generate the next available versioned directory path.

    Prevents overwriting existing result directories by
    incrementing a version number.
    """
    if not base_path.exists():
        return base_path

    parent = base_path.parent
    base_name = base_path.name

    # Find existing versioned directories
    existing_dirs = [d for d in parent.glob(f"{base_name}*") if d.is_dir()]

    versions = [0]
    for dir_path in existing_dirs:
        dir_name = dir_path.name
        if dir_name == base_name:
            continue
        try:
            version = int(dir_name.replace(f"{base_name}-", ""))
            versions.append(version)
        except ValueError:
            continue

    next_version = max(versions) + 1
    return parent / f"{base_name}-{next_version}"

# filtering the outliers based on the ratio of a bounding box area with the image area
def filter_detections(detections, image_path, max_relative_area=0.0035, min_confidence=0.6): # reduced max_relative_area from 0.4 (40%) to 0.0035 (0.35%)
    # get numbers
    #print("\n", "type(detections): ", type(detections))
    #print("len(detections): ", len(detections), "\n")

    # get attributes
    #print("\ndetections attributes: ", dir(detections))
    #print("\ndetections[0] attributes: ", dir(detections[0]))
    #print("\ndetections[0].bbox attributes: ", dir(detections[0].bbox))

    """
    Robust detection filtering with image dimension handling.

    Filters detections based on:
    - Relative bounding box area
    - Confidence threshold
    - Direct image dimension reading
    """
    try:
        # Safely read image dimensions
        with Image.open(image_path) as img:
            img_width, img_height = img.size

        filtered_detections = []

        for detection in detections:
            # Safely extract bounding box coordinates
            bbox = detection.bbox.to_xyxy()

            # Robust dimension and confidence calculation
            bbox_width = max(0, bbox[2] - bbox[0])
            bbox_height = max(0, bbox[3] - bbox[1])

            # Prevent division by zero
            relative_area = (
                (bbox_width * bbox_height) / (img_width * img_height)
                if img_width > 0 and img_height > 0
                else 0
            )

            # Safe confidence extraction
            try:
                confidence = float(detection.score.value)
            except (TypeError, AttributeError):
                confidence = 0.0

            # Apply filtering criteria with detailed logging
            if (relative_area <= max_relative_area and
                confidence >= min_confidence):
                filtered_detections.append(detection)
            else:
                logger.debug(
                    f"Filtered detection: "
                    f"Area={relative_area:.4f}, "
                    f"Confidence={confidence:.4f}"
                )

        # get numbers
        #print("\n", "type(filtered_detections): ", type(filtered_detections))
        #print("len(filtered_detections): ", len(filtered_detections), "\n")

        return filtered_detections

    except Exception as e:
        logger.error(f"Error in filter_detections: {e}")
        return []

# converting between the two suitable file types for saving the resulting outputs
def convert_yolo_to_labelme(image_path, detections, class_names):
    """
    Comprehensive conversion of YOLO detections to LabelMe format.

    Includes:
    - Robust error handling
    - Full conversion of all detections
    - Detailed JSON generation
    """
    try:
        # Open image safely
        with Image.open(image_path) as img:
            img_width, img_height = img.size

        # Prepare shapes with comprehensive type checking
        shapes = []
        for detection in detections:
            try:
                # Safe attribute extraction
                class_id = int(detection.category.id)
                bbox = detection.bbox.to_xyxy()
                confidence = float(detection.score.value)

                shape = {
                    "label": str(
                        class_names.get(
                            class_id,
                            f"Unknown_Class_{class_id}"
                        )
                    ),
                    "points": [
                        [float(bbox[0]), float(bbox[1])],  # Top-left
                        [float(bbox[2]), float(bbox[1])],  # Top-right
                        [float(bbox[2]), float(bbox[3])],  # Bottom-right
                        [float(bbox[0]), float(bbox[3])]   # Bottom-left
                    ],
                    "group_id": None,
                    "shape_type": "polygon",
                    "flags": {},
                    "confidence": confidence
                }
                shapes.append(shape)

            except Exception as detection_err:
                logger.error(
                    f"Error processing detection in conversion: {detection_err}"
                )

        # Construct LabelMe format dictionary
        labelme_format = {
            "version": "5.1.1",
            "flags": {},
            "shapes": shapes,
            "imagePath": os.path.basename(image_path),
            "imageData": None,
            "imageHeight": int(img_height),
            "imageWidth": int(img_width)
        }

        return labelme_format

    except Exception as main_err:
        logger.error(f"LabelMe conversion error for {image_path}: {main_err}")
        return None

# filtering the outliers based on the average bounding box area using statistical techniques
def remove_area_outliers(detections, multiplier=4.5): # increased multiplier from 1.5 (strict) to 4.5 (lenient)
    """
    Remove area outliers using the IQR method

    Args:
        detections (list): List of object predictions
        multiplier (float): IQR multiplier for outlier detection (new [lenient] default = 3.0, old [strict] default = 1.5)

    Returns:
        list: Filtered detections without area outliers
    """
    # Extract areas
    areas = [detection.bbox.area for detection in detections]

    # Calculate Q1, Q3, and IQR
    q1 = np.percentile(areas, 25)
    q3 = np.percentile(areas, 75)
    iqr = q3 - q1

    # Define outlier boundaries
    lower_bound = q1 - (multiplier * iqr)
    upper_bound = q3 + (multiplier * iqr)

    # Filter detections
    filtered_detections = [
        detection for detection in detections
        if lower_bound <= detection.bbox.area <= upper_bound
    ]

    # Log outlier information
    logger.info(f"Total detections: {len(detections)}")
    logger.info(f"Detections after outlier removal: {len(filtered_detections)}")
    logger.info(f"Outlier boundaries: [{lower_bound:.2f}, {upper_bound:.2f}]")
    logger.info(f"Removed {len(detections) - len(filtered_detections)} outliers")

    return filtered_detections

# visualizing area distribution before and after filtering
def plot_area_distribution(original_detections, filtered_detections):
    original_areas = [det.bbox.area for det in original_detections]
    filtered_areas = [det.bbox.area for det in filtered_detections]

    plt.figure(figsize=(14, 6))

    # Box plot for original areas
    plt.subplot(2, 2, 1)
    plt.title('Input Areas - With Outliers - Box Plot')
    sns.boxplot(original_areas)
    plt.xlabel('Area')

    # Box plot for filtered areas
    plt.subplot(2, 2, 2)
    plt.title('Output Areas - Without Outliers - Box Plot')
    sns.boxplot(filtered_areas)
    plt.xlabel('Area')

    # Violin plot for original areas
    plt.subplot(2, 2, 3)
    plt.title('Input Areas - With Outliers - Violin Plot')
    sns.violinplot(original_areas)
    plt.xlabel('Area')

    # Violin plot for filtered areas
    plt.subplot(2, 2, 4)
    plt.title('Output Areas - Without Outliers - Violin Plot')
    sns.violinplot(filtered_areas)
    plt.xlabel('Area')

    plt.tight_layout()
    plt.savefig('area_distribution_comparison.png')
    plt.show()

# using the trained model to detect objects in images
def test_sahi_yolo(
    model_path,
    test_images_dir,
    project_dir,
    name='detection',
    slice_height=640,  # maybe reduce from 640 to 512/480
    slice_width=640,   # maybe reduce from 640 to 512/480
    overlap_ratio=0.2, # maybe increase from 0.2 to 0.3
    conf_threshold=0.5 # maybe increase from 0.5 to 0.6
):
    """
    Enhanced object detection pipeline with:
    - Comprehensive error handling
    - Detailed logging
    - Performance tracking
    """
    start_time = datetime.now()
    logger.info(f"Starting detection process at {start_time}")
    logger.info(f"Model Path: {model_path}")
    logger.info(f"Test Images Directory: {test_images_dir}")
    logger.info(f"Project Directory: {project_dir}")

    # Setup output directories
    base_gen_lab_dir = Path(project_dir) / "generated-labeling"
    gen_lab_dir = get_next_version_path(base_gen_lab_dir)
    yolo_dir = gen_lab_dir / "yolo"
    labelme_dir = gen_lab_dir / "labelme"

    # Create output directories
    for directory in [gen_lab_dir, yolo_dir, labelme_dir]:
        directory.mkdir(parents=True, exist_ok=True)
    logger.info(f"Created output directories in: {gen_lab_dir}")

    # Initialize detection model
    try:
        detection_model = AutoDetectionModel.from_pretrained(
            model_type='yolov8',
            model_path=model_path,
            confidence_threshold=conf_threshold
        )
        logger.info("YOLO model loaded successfully")
    except Exception as model_load_err:
        logger.error(f"Model loading failed: {model_load_err}")
        return

    # Performance tracking variables
    total_images = 0
    total_detections = 0
    failed_images = []
    detailed_errors = {}

    # Walk through image directories
    for root, _, files in os.walk(test_images_dir):
        image_files = [
            f for f in files
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]

        if not image_files:
            continue

        # Create corresponding output subdirectories
        relative_path = os.path.relpath(root, test_images_dir)
        current_yolo_dir = yolo_dir / relative_path
        current_labelme_dir = labelme_dir / relative_path
        current_yolo_dir.mkdir(parents=True, exist_ok=True)
        current_labelme_dir.mkdir(parents=True, exist_ok=True)

        logger.info(f"Processing directory: {relative_path}")

        # Process each image
        for image_name in tqdm(image_files, desc=f"Processing {relative_path}", unit="image"):
            try:
                image_path = os.path.join(root, image_name)
                logger.info(f"Processing image: {image_path}")

                # Perform sliced prediction
                results = get_sliced_prediction(
                    image_path,
                    detection_model,
                    slice_height=slice_height,
                    slice_width=slice_width,
                    overlap_height_ratio=overlap_ratio,
                    overlap_width_ratio=overlap_ratio
                )

                # get numbers
                #print("\n", "type(results): ", type(results))
                print("\n", "type(results.object_prediction_list): ", type(results.object_prediction_list))
                print("len(results.object_prediction_list): ", len(results.object_prediction_list), "\n")

                # get attributes
                #print("\nresults attributes: ", dir(results))
                #print("\nresults.object_prediction_list attributes: ", dir(results.object_prediction_list))
                #print("\nresults.object_prediction_list[0] attributes: ", dir(results.object_prediction_list[0]))
                #print("\nresults.object_prediction_list[0].bbox attributes: ", dir(results.object_prediction_list[0].bbox))

                # Filter detections (as a list)
                filtered_results = filter_detections(
                    results.object_prediction_list,
                    image_path,
                    max_relative_area=0.0035, # reduced max_relative_area from 0.4 (40%) to 0.0035 (0.35%)
                    min_confidence=conf_threshold
                )

                # get numbers
                print("\n", "type(filtered_results): ", type(filtered_results))
                print("len(filtered_results): ", len(filtered_results), "\n")

                # visualize area distribution before and after filtering
                print("\n> plotting \'results.object_prediction_list\' & \'filtered_results\' values >\n")
                plot_area_distribution(results.object_prediction_list, filtered_results) # (original_detections, filtered_detections)

                # filter out outliers based on bounding box area using statistical techniques
                cleaned_results = remove_area_outliers(filtered_results)

                # get numbers
                print("\n", "type(cleaned_results): ", type(cleaned_results))
                print("len(cleaned_results): ", len(cleaned_results), "\n")

                # visualize area distribution before and after filtering
                print("\n> plotting \'filtered_results\' & \'cleaned_results\' values >\n")
                plot_area_distribution(filtered_results, cleaned_results) # (original_detections, filtered_detections)

                # Update tracking
                num_detections = len(cleaned_results) # replaced "filtered_results" with "cleaned_results" to remove large outliers
                total_images += 1
                total_detections += num_detections

                # Save YOLO format
                yolo_output_path = current_yolo_dir / f"{os.path.splitext(image_name)[0]}.txt"
                with open(yolo_output_path, 'w') as f:
                    for pred in cleaned_results: # replaced "filtered_results" with "cleaned_results" to remove large outliers
                        bbox = pred.bbox.to_xyxy()
                        class_id = pred.category.id
                        conf = float(pred.score.value)
                        f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]} {conf}\n")

                # Save LabelMe format results
                labelme_json = convert_yolo_to_labelme(
                    image_path,
                    cleaned_results, # replaced "filtered_results" with "cleaned_results" to remove large outliers
                    detection_model.model.names
                )

                if labelme_json:
                    labelme_output_path = current_labelme_dir / f"{os.path.splitext(image_name)[0]}.json"
                    with open(labelme_output_path, 'w') as f:
                        json.dump(labelme_json, f, indent=2)
                else:
                    # Track detailed error information
                    detailed_errors[image_path] = "LabelMe JSON conversion failed"
                    failed_images.append(image_path)

                logger.info(f"Completed {image_name}: {num_detections} detections")

            except Exception as e:
                logger.error(f"Comprehensive error processing {image_path}:")
                logger.error(f"Error type: {type(e)}")
                logger.error(f"Error details: {str(e)}")

                # Track detailed error information
                detailed_errors[image_path] = str(e)
                failed_images.append(image_path)
                continue

    # Final reporting
    end_time = datetime.now()
    processing_time = end_time - start_time

    logger.info("\nDetection Process Summary:")
    logger.info(f"Total processing time: {processing_time}")
    logger.info(f"Total images processed: {total_images}")
    logger.info(f"Total detections found: {total_detections}")
    logger.info(f"Average detections per image: {total_detections/total_images:.2f}" if total_images > 0 else "No images processed")

    if failed_images:
        logger.warning(f"Failed to process {len(failed_images)} images")
        for img, error in detailed_errors.items():
            logger.warning(f"Image: {img} - Error: {error}")

    logger.info(f"\nResults saved in:")
    logger.info(f"YOLO format: {yolo_dir}")
    logger.info(f"LabelMe format: {labelme_dir}")

# using the google drive paths from the original script
project_input = "/content/drive/MyDrive/October15.v1i.yolov8"
result_input = f'{project_input}/results'
run_input = "200_epochs-"
model_input = f'{result_input}/{run_input}/weights/best.pt'
test_input = f'{project_input}/sampling_set/'

# running the execution step code block
if __name__ == "__main__":
    # Verify paths before running
    logger.info("Verifying paths...")
    logger.info(f"Model path: {model_input}")
    logger.info(f"Test images directory: {test_input}")

    if not os.path.exists(model_input):
        raise FileNotFoundError(f"Model file not found at: {model_input}")

    # Run detection with full configuration
    test_sahi_yolo(
        model_path=model_input,
        test_images_dir=test_input,
        project_dir=result_input
    )

# The end.